| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

#### This notebook is licensed under GPL 3.0. Please visit our Github repo for more information: 
#### The notebook was collaboratively authored by the Environmental Data & Governance Initiative (EDGI) following our authorship protocol: https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/
#### For more information about this project, visit https://www.environmentalenforcementwatch.org/

## How to Run this Notebook
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---

# Nationwide statistics about environmental compliance trends

## Setup
Here we load some helper code to get us going.

In [ ]:
# Import code libraries
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git &>/dev/null;
#!git clone https://github.com/edgi-govdata-archiving/ECHO-COVID19.git &>/dev/null;
#%run ECHO-COVID19/utilities.py don't need this if not charting?
%run ECHO_modules/DataSet.py

import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import csv
import datetime
import folium
from folium.plugins import FastMarkerCluster
import ipywidgets as widgets
from IPython.core.display import display, HTML
from pandas.errors import EmptyDataError
def formatter(value):
  return "{:0.2f}".format(value)

Here we set up some code to help us store and eventually export the metrics.

In [ ]:
inspections = dict()
violations = dict()
enforcements = dict()
penalties = dict()
emissions = dict()

## Start getting data
First, get summary data from the ECHO_EXPORTER table.

In [ ]:
# Get everything we will need from ECHO_EXPORTER in a single DB query.
# We can then use the full dataframe to specialize views of it.
full_echo_data = None
column_mapping = {
    '"REGISTRY_ID"': str,
    '"FAC_NAME"': str,
    '"FAC_LAT"': float,
    '"FAC_LONG"': float,
    '"AIR_IDS"': str,
    '"NPDES_IDS"': str,
    '"RCRA_IDS"': str,
    '"DFR_URL"': str,
    '"AIR_FLAG"': str,
    '"NPDES_FLAG"': str,
    '"GHG_FLAG"': str,
    '"RCRA_FLAG"': str,
    '"FAC_ACTIVE_FLAG"': str
}
column_names = list( column_mapping.keys() )
columns_string = ','.join( column_names )
sql = 'select ' + columns_string + ' from "ECHO_EXPORTER" where "AIR_FLAG" = \'Y\' or "NPDES_FLAG" = \'Y\' or "GHG_FLAG" = \'Y\' or "RCRA_FLAG" = \'Y\''
try:
    # Don't index.
    full_echo_data = get_data( sql )
except EmptyDataError:
    print("\nThere are no EPA facilities for this query.\n")
full_echo_data

,REGISTRY_ID,FAC_NAME,FAC_LAT,FAC_LONG,AIR_IDS,NPDES_IDS,RCRA_IDS,DFR_URL,AIR_FLAG,NPDES_FLAG,GHG_FLAG,RCRA_FLAG,FAC_ACTIVE_FLAG
0,1.100079e+11,FRANKLINVILLE CENTRAL OFFICE,39.619710,-75.082060,NaN,NaN,NJD980645550,http://echo.epa.gov/detailed-facility-report?f...,N,N,N,Y,NaN
1,1.100042e+11,INSTRUMENTAL ENGINEERING,41.023599,-74.202748,NaN,NaN,NJD089751200,http://echo.epa.gov/detailed-facility-report?f...,N,N,N,Y,NaN
2,1.100047e+11,MIKE BASS USED CAR,41.451417,-82.166056,NaN,NaN,OHD987049517,http://echo.epa.gov/detailed-facility-report?f...,N,N,N,Y,NaN
3,1.100706e+11,PALM PLAZA,28.818830,-81.887510,NaN,FLR10SZ00,NaN,http://echo.epa.gov/detailed-facility-report?f...,N,Y,N,N,Y
4,1.100202e+11,WILDLIFE RECREATION POND 1,32.247591,-87.791091,NaN,ALR165713,NaN,http://echo.epa.gov/detailed-facility-report?f...,N,Y,N,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852340,1.100704e+11,PARKS AT LEGACY PHASE 2,33.229586,-96.850452,NaN,TXR15900B,NaN,http://echo.epa.gov/detailed-facility-report?f...,N,Y,N,N,Y
1852341,1.100052e+11,BERNHARDT & TAYLOR CUSTOM CABINETS,37.502910,-77.601800,NaN,NaN,VAD003117561,http://echo.epa.gov/detailed-facility-report?f...,N,N,N,Y,Y
1852342,1.100556e+11,FOUNDATION ENERGY MGMT - SCHRAMM #1,40.077161,-102.593594,CO0000000812502009,NaN,NaN,http://echo.epa.gov/detailed-facility-report?f...,Y,N,N,N,NaN
1852343,1.100201e+11,FLOYD PIT,31.771450,-86.007790,NaN,ALG890136 ALR161820,NaN,http://echo.epa.gov/detailed-facility-report?f...,N,Y,N,N,Y


## Number of Currently Regulated Facilities Per Program

In [ ]:
air_fac = full_echo_data.loc[(full_echo_data["AIR_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]
water_fac = full_echo_data.loc[(full_echo_data["NPDES_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]
waste_fac = full_echo_data.loc[(full_echo_data["RCRA_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]
ghg_fac = full_echo_data.loc[(full_echo_data["GHG_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]

display(HTML("<h3>There are "+ str(air_fac) + " facilities currently regulated under the Clean Air Act.</h3>"))
display(HTML("<h3>There are "+ str(water_fac) + " facilities currently regulated under the Clean Water Act.</h3>"))
display(HTML("<h3>There are "+ str(waste_fac) + " facilities currently regulated under RCRA (hazardous waste).</h3>"))
display(HTML("<h3>There are "+ str(ghg_fac) + " facilities currently reporting greenhouse gas emissions.</h3>"))

## Clean Air Act inspections in 2019

In [ ]:
# Use SQL to search for and select the data about air stack tests
air_inspections = None
try:
    sql = 'select * from \"ICIS-AIR_FCES_PCES\" where \"ACTUAL_END_DATE\" like \'__-__-2019\''

    # Download the data from that URL
    air_inspections = get_data( sql, 'pgm_sys_id' )
except EmptyDataError:
    print( "No data found")

air_inspections

,PGM_SYS_ID,ACTIVITY_ID,STATE_EPA_FLAG,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,COMP_MONITOR_TYPE_CODE,COMP_MONITOR_TYPE_DESC,ACTUAL_END_DATE,PROGRAM_CODES
0,020000003606390000,3601943049,E,INS,Inspection/Evaluation,PCE,PCE On-Site,07-23-2019,"CAACFC, CAAFESOP"
1,020000003606501000,3601851095,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,05-08-2019,CAAOP
2,020000003606501000,3601866216,E,INS,Inspection/Evaluation,FOO,FCE On-Site,05-01-2019,"CAANAM, CAAOP"
3,020000003606501000,3601972076,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,04-24-2019,CAASIP
4,020000003606501000,3601972077,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,04-17-2019,CAASIP
...,...,...,...,...,...,...,...,...,...
48848,VA0000005108900033,3601738402,S,INS,Inspection/Evaluation,PFF,PCE Off-Site,01-23-2019,CAAMACT
48849,VA0000005108900033,3601744587,S,INS,Inspection/Evaluation,PFF,PCE Off-Site,02-07-2019,CAATVP
48850,VA0000005108900033,3601912786,S,INS,Inspection/Evaluation,PFF,PCE Off-Site,07-19-2019,"CAAMACT, CAASIP, CAATVP"
48851,VA0000005108900033,3601955529,S,INS,Inspection/Evaluation,PFF,PCE Off-Site,08-22-2019,CAATVP


In [ ]:
# Number of inspections in 2019 per 1000 regulated facilities

air_inspections_metric = formatter((air_inspections.shape[0] / air_fac) * 1000) # Divide by regulated facilities and multiply by desired rate (per 1000)
inspections["CAA"] = air_inspections_metric
display(HTML("<h3>"+ air_inspections_metric +" inspections per 1000 facilities</h3>"))

## High priority violations of the Clean Air Act in 2019



In [ ]:
air_violations = None
try:
    sql = 'select * from "ICIS-AIR_VIOLATION_HISTORY" where "HPV_DAYZERO_DATE" like \'__-__-2019\''

    air_violations = get_data( sql, "pgm_sys_id" )

    # Remove "FACIL" violations, which are paperwork violations according to: https://19january2017snapshot.epa.gov/sites/production/files/2013-10/documents/frvmemo.pdf
    # air_violations = air_violations.loc[(air_violations["POLLUTANT_DESCS"]!="FACIL")]
except EmptyDataError:
    print( "No data found")
air_violations

,PGM_SYS_ID,ACTIVITY_ID,AGENCY_TYPE_DESC,STATE_CODE,AIR_LCON_CODE,COMP_DETERMINATION_UID,ENF_RESPONSE_POLICY_CODE,PROGRAM_CODES,PROGRAM_DESCS,POLLUTANT_CODES,POLLUTANT_DESCS,EARLIEST_FRV_DETERM_DATE,HPV_DAYZERO_DATE,HPV_RESOLVED_DATE
0,IN0000001803300043,3601685463,State,IN,NaN,IN000A74118,HPV,CAANSPS,New Source Performance Standards,10373,Particulate matter - PM10,12-03-2018,01-13-2019,NaN
1,IN0000001801900008,3601799804,State,IN,NaN,IN000A75598,HPV,CAAMACT,MACT Standards (40 CFR Part 63),300000036,Mercury,04-05-2019,04-30-2019,05-26-2020
2,IN0000001801900008,3602022157,State,IN,NaN,IN000A78256,HPV,CAAMACT,MACT Standards (40 CFR Part 63),300000036,Mercury,09-25-2019,12-24-2019,NaN
3,MN0000002704700055,3602241949,State,MN,NaN,MN000A00001670PEN20191,HPV,CAATVP,Title V Permits,10193,Carbon monoxide,06-06-2018,04-23-2019,06-04-2020
4,PA000493288,3601889184,State,PA,NaN,PA000A0000H00000000376058,HPV,CAAMACT CAANSPS CAASIP CAATVP,MACT Standards (40 CFR Part 63) New Source Per...,10358,Nitrogen oxides,06-06-2019,06-06-2019,04-23-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,IN0000001801900155,3602134543,State,IN,NaN,IN000A80006,HPV,CAATVP,Title V Permits,300000018,Hydrochloric acid,12-16-2019,12-16-2019,NaN
814,WAORC005302700058,3602032106,Local,WA,ORC,WAORCA78445,HPV,CAATVP,Title V Permits,300000243,VOLATILE ORGANIC COMPOUNDS (VOCS),10-29-2019,10-29-2019,NaN
815,CO0000000812309FC0,3602053305,State,CO,NaN,CO000A00000812309FC000001,HPV,CAASIP,State Implementation Plan for National Primary...,300000243,VOLATILE ORGANIC COMPOUNDS (VOCS),06-25-2019,06-25-2019,10-03-2019
816,AL0000000104900048,3602051772,State,AL,NaN,AL000A78887,HPV,CAATVP,Title V Permits,300000328,ADMIN,11-05-2019,11-05-2019,NaN


In [ ]:
# Number of high priority violations per 1000 regulated facilities

#air_violations_fac = air_violations.shape[0] / len(air_violations["PGM_SYS_ID"].unique()) # Total number of violations divided by number of facilities with violations. Will use this later in looking at enforcement actions.
air_violations_metric = formatter((air_violations.shape[0] / air_fac) * 1000) # Divide by regulated facilities and multiply by desired rate (per 1000)
violations["CAA"] = air_violations_metric
display(HTML("<h3>"+air_violations_metric+" high priority violations per 1000 facilities </h3>"))

## Formal Enforcement Actions and Penalties under the Clean Air Act in 2019

In [ ]:
air_enforcements = None
try:
    sql = 'select * from "ICIS-AIR_FORMAL_ACTIONS" where "SETTLEMENT_ENTERED_DATE" like \'__/__/2019\''

    air_enforcements = get_data( sql, "pgm_sys_id" )
except EmptyDataError:
    print( "No data found")
air_enforcements

,PGM_SYS_ID,ACTIVITY_ID,ENF_IDENTIFIER,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,STATE_EPA_FLAG,ENF_TYPE_CODE,ENF_TYPE_DESC,SETTLEMENT_ENTERED_DATE,PENALTY_AMOUNT
0,OH0000000627010056,31941,05-1999-0644,JDC,Judicial,E,CIV,Civil Judicial Action,07/17/2019,0.0
1,OH0000000684000000,31941,05-1999-0644,JDC,Judicial,E,CIV,Civil Judicial Action,07/17/2019,0.0
2,IN0000001802900002,31941,05-1999-0644,JDC,Judicial,E,CIV,Civil Judicial Action,07/17/2019,0.0
3,IN0000001814700020,31941,05-1999-0644,JDC,Judicial,E,CIV,Civil Judicial Action,07/17/2019,0.0
4,OH0000000165000006,31941,05-1999-0644,JDC,Judicial,E,CIV,Civil Judicial Action,07/17/2019,0.0
...,...,...,...,...,...,...,...,...,...,...
2636,LA0000002202300278,3602222676,LA000A3059731,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,08/13/2019,0.0
2637,LA0000002205700094,3602222677,LA000A3059701,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,08/13/2019,0.0
2638,LA0000002205900097,3602232677,LA000A2863864,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,11/14/2019,0.0
2639,HI0000001500700066,3602236126,HI000AEA93,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,07/11/2019,22800.0


In [ ]:
# Number of formal actions in 2019 per violation

air_enforcements_metric = formatter(air_enforcements.shape[0]/air_violations.shape[0]) # Formal actions divided by number of violations
enforcements["CAA"] = air_enforcements_metric
display(HTML("<h3>"+air_enforcements_metric +" formal enforcement actions per violation</h3>"))

In [ ]:
air_enforcements["ENF_TYPE_DESC"].unique()

array(['Civil Judicial Action', 'Pre-Referral Negotiation',
       'Administrative Order', 'CAA 113D1 Action For Penalty',
       'CAA 113 Notice Of Violation',
       'CAA 113A Admin Compliance Order (Non-Penalty)',
       'CAA 120 AO For Noncompliance Penalty',
       'CAA 303 AO For Imminent Hazard',
       'CAA 112R9 AO For Imm Hazard/Accdntl Release',
       'CAA 113D1 Action For Penalty - 112(r) Expedited Settlement Program',
       'CAA 205 Action For Mobile/Title II Expedited Settlement Program',
       'Access Order',
       'CAA 205 Action For Vehicles and Engines Defeat Device/Tampering Expedited Settlement Program'],
      dtype=object)

In [ ]:
# Penalties each year per penalized facility
air_penalties = air_enforcements.loc[air_enforcements["PENALTY_AMOUNT"]>0]
air_penalties_metric = formatter(sum(air_penalties["PENALTY_AMOUNT"]) / len(air_penalties["PGM_SYS_ID"].unique())) #Divide the sum of penalties by number of penalized facilities
penalties["CAA"] = air_penalties_metric
display(HTML("<h3>$"+air_penalties_metric +" per penalized facility</h3>"))

---

## Clean Water Act inspections in 2019

In [ ]:
# Find facilities with pollutant exceedences
water_inspections = None
try:
    sql = 'select "NPDES_ID", "REGISTRY_ID", "ACTUAL_END_DATE", "STATE_EPA_FLAG"' + \
        ' from "NPDES_INSPECTIONS" where "ACTUAL_END_DATE" like \'__/__/2019\''

    water_inspections = get_data( sql, "NPDES_ID" ) 
except EmptyDataError:
    print( "No data found")
water_inspections

,REGISTRY_ID,ACTUAL_END_DATE,STATE_EPA_FLAG
NPDES_ID,,,
AK0000507,110030488620,12/17/2019,S
AK0021385,110000761453,08/26/2019,E
AK0022497,110039730459,02/25/2019,S
AKG370029,110028064387,08/21/2019,S
AKG520042,110064604094,06/07/2019,S
...,...,...,...
WYR320686,110055196746,11/07/2019,S
WYR320871,110070235311,03/25/2019,S
WY0095885,110069994811,05/02/2019,S


In [ ]:
# Number of inspections in 2019 per 1000 regulated facilities

water_inspections_metric = formatter((water_inspections.shape[0] / water_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
inspections["CWA"] = water_inspections_metric
display(HTML("<h3>"+water_inspections_metric +" inspections per 1000 facilities</h3>"))

## Effluent violations of the Clean Water Act in 2019
*NOTE*: Not other kind of violations (schedule, permit, single event)

In [ ]:
# Find facilities with pollutant exceedences
water_violations = None
try:
    sql = 'select "NPDES_ID", "EXCEEDENCE_PCT", "MONITORING_PERIOD_END_DATE", "VIOLATION_CODE", "PARAMETER_DESC"' + \
        ' from "NPDES_EFF_VIOLATIONS" where "VIOLATION_CODE" like \'E90\' and "MONITORING_PERIOD_END_DATE" like \'__/__/2019\''
    water_violations = get_data( sql, "NPDES_ID" ) 
except EmptyDataError:
    print( "No data found")
water_violations

,EXCEEDENCE_PCT,MONITORING_PERIOD_END_DATE,VIOLATION_CODE,PARAMETER_DESC
NPDES_ID,,,,
IN0039497,173.0,08/31/2019,E90,"Nitrogen, ammonia total [as N]"
SC0031801,235.0,04/30/2019,E90,"Nitrogen, ammonia total [as N]"
NC0023884,5.0,02/28/2019,E90,"Flow, in conduit or thru treatment plant"
AR0022756,100.0,08/31/2019,E90,Whole Effluent Toxicity [WET] - D. pulex
OR0022721,261.0,03/31/2019,E90,"Nitrogen, ammonia total [as N]"
...,...,...,...,...
TX0083003,13.0,06/30/2019,E90,"Solids, total suspended"
AL0081396,119.0,03/31/2019,E90,"Nitrogen, ammonia total [as N]"
LA0126276,20.0,06/30/2019,E90,"Solids, total suspended"


In [ ]:
# Number of violations each year per 1000 regulated facilities

water_violations_metric = formatter((water_violations.shape[0] / water_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
violations["CWA"] = water_violations_metric
display(HTML("<h3>"+water_violations_metric+" violations per 1000 facilities</h3>"))

## Enforcement Actions and Penalties under the Clean Water Act in 2019

In [ ]:
# Find facilities with pollutant exceedences
water_enforcements = None
try:
    sql = 'select "NPDES_ID", "AGENCY", "ENF_TYPE_DESC", "SETTLEMENT_ENTERED_DATE", "FED_PENALTY_ASSESSED_AMT", "STATE_LOCAL_PENALTY_AMT"' + \
        ' from "NPDES_FORMAL_ENFORCEMENT_ACTIONS" where "SETTLEMENT_ENTERED_DATE" like \'__/__/2019\''

    water_enforcements = get_data( sql, "NPDES_ID" ) 
except EmptyDataError:
    print( "No data found")
water_enforcements

,AGENCY,ENF_TYPE_DESC,SETTLEMENT_ENTERED_DATE,FED_PENALTY_ASSESSED_AMT,STATE_LOCAL_PENALTY_AMT
NPDES_ID,,,,,
AL0032310,EPA,CWA 309G2B AO For Class II Penalties,10/28/2019,50000.0,NaN
AR0041327,State,State Administrative Order of Consent,02/25/2019,NaN,4200.0
CA0037770,State,State CWA Penalty AO,05/16/2019,NaN,NaN
CA0048127,State,State CWA Penalty AO,05/21/2019,NaN,NaN
CA0059188,State,State CWA Penalty AO,03/01/2019,NaN,NaN
...,...,...,...,...,...
PAS212211,State,State CWA Non Penalty AO,01/03/2019,NaN,NaN
SC0040606,State,State Administrative Order of Consent,05/24/2019,NaN,90.0
TXR05EJ33,State,State CWA Penalty AO,09/10/2019,NaN,875.0


In [ ]:
# Number of formal actions in 2019 per violation
water_enforcements_metric = formatter(water_enforcements.shape[0]/water_violations.shape[0]) # Formal actions divided by number of violations
enforcements["CWA"] = water_enforcements_metric
display(HTML("<h3>"+water_enforcements_metric +" formal enforcement actions per violation</h3>"))

In [ ]:
water_enforcements["ENF_TYPE_DESC"].unique()

array(['CWA 309G2B AO For Class II Penalties',
       'State Administrative Order of Consent', 'State CWA Penalty AO',
       'CWA 309A AO For Compliance', 'State CWA Non Penalty AO',
       'Civil Judicial Action', 'CWA 309G2A AO For Class I Penalties',
       'RCRA 3008A AO For Comp And/Or Penalty',
       'CWA 309G2E AO For Class I Penalties - Storm Water Construction Expedited Settlement Program',
       'CWA 311B6B1 AO For Class I Penalty - SPCC Expedited Settlement Program',
       'Order of Suspension or Revocation'], dtype=object)

In [ ]:
# Penalties each year per penalized facility
water_penalties = water_enforcements.loc[water_enforcements["FED_PENALTY_ASSESSED_AMT"]>0]
water_penalties_metric = formatter(sum(water_penalties["FED_PENALTY_ASSESSED_AMT"]) / len(water_penalties.index.unique())) #Divide the sum of penalties by number of penalized facilities
penalties["CWA"] = water_penalties_metric
display(HTML("<h3>$"+water_penalties_metric +" per penalized facility</h3>"))

## RCRA inspections in 2019

In [ ]:
# Find facilities with pollutant exceedences
waste_inspections = None
try:
    sql = 'select * from "RCRA_EVALUATIONS" where "EVALUATION_START_DATE" like \'__/__/2019\''

    waste_inspections = get_data( sql, "ID_NUMBER" ) 
except EmptyDataError:
    print( "No data found")
waste_inspections

,ACTIVITY_LOCATION,EVALUATION_IDENTIFIER,EVALUATION_TYPE,EVALUATION_DESC,EVALUATION_AGENCY,EVALUATION_START_DATE,FOUND_VIOLATION
ID_NUMBER,,,,,,,
MAD982196164,MA,001,CEI,COMPLIANCE EVALUATION INSPECTION ON-SITE,S,08/07/2019,N
WIR000170340,WI,001,FCI,FOCUSED COMPLIANCE INSPECTION,S,07/02/2019,Y
WID006129225,WI,001,CEI,COMPLIANCE EVALUATION INSPECTION ON-SITE,S,10/07/2019,Y
KYR000056879,KY,001,NRR,NON-FINANCIAL RECORD REVIEW,S,10/11/2019,N
KYD006376347,KY,001,CEI,COMPLIANCE EVALUATION INSPECTION ON-SITE,S,09/25/2019,N
...,...,...,...,...,...,...,...
OHR000210906,OH,001,FCI,FOCUSED COMPLIANCE INSPECTION,S,03/18/2019,Y
RID987470598,RI,001,CSE,COMPLIANCE SCHEDULE EVALUATION,S,03/20/2019,N
KYR000049049,KY,001,CEI,COMPLIANCE EVALUATION INSPECTION ON-SITE,S,03/12/2019,N


In [ ]:
# Number of inspections in 2019 per 1000 regulated facilities
waste_inspections_metric = formatter((waste_inspections.shape[0] / waste_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
inspections["RCRA"] = waste_inspections_metric
display(HTML("<h3>"+waste_inspections_metric+" inspections per 1000 facilities</h3>"))

## Violations of RCRA in 2019

In [ ]:
# Find facilities with pollutant exceedences
waste_violations = None
try:
    sql = 'select * from "RCRA_VIOLATIONS" where "DATE_VIOLATION_DETERMINED" like \'__/__/2019\''

    waste_violations = get_data( sql, "ID_NUMBER" ) 
except EmptyDataError:
    print( "No data found")
waste_violations

,ACTIVITY_LOCATION,VIOLATION_TYPE,VIOLATION_TYPE_DESC,VIOL_DETERMINED_BY_AGENCY,DATE_VIOLATION_DETERMINED,ACTUAL_RTC_DATE,SCHEDULED_COMPLIANCE_DATE
ID_NUMBER,,,,,,,
MID006020275,MI,262.D,Standards Applicable to Recordkeeping and Repo...,S,11/25/2019,NaN,NaN
DEN201500044,DE,261.A,ID and Listing of HW: General,S,05/22/2019,05/22/2019,NaN
SCD069316271,SC,273.B,Standards for Universal Waste Management: Stan...,S,12/17/2019,03/03/2020,NaN
MAD985294750,MA,262.A,Standards Applicable to Generators of HW: General,S,11/15/2019,01/21/2020,NaN
CTD983871930,CT,262.C,Standards Applicable to Generators of HW: Pre-...,S,04/25/2019,06/18/2019,NaN
...,...,...,...,...,...,...,...
TXD981612989,TX,262.D,Standards Applicable to Recordkeeping and Repo...,S,09/11/2019,12/12/2019,NaN
ORD131968380,OR,262.C,Standards Applicable to Generators of HW: Pre-...,S,04/09/2019,04/17/2019,NaN
MID980615298,MI,264.C,Standards for Owners and Operators of HW TSDs:...,S,06/26/2019,NaN,NaN


In [ ]:
# Number of violations in 2019 per 1000 regulated facilities
waste_violations_metric = formatter((waste_violations.shape[0] / waste_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
violations["RCRA"] = waste_violations_metric
display(HTML("<h3>"+waste_violations_metric+" violations per 1000 facilities</h3>"))

## Enforcement Actions and Penalties under RCRA in 2019

In [ ]:
# Find facilities with enforcement actions
waste_enforcements = None
try:
    sql = 'select * from "RCRA_ENFORCEMENTS" where "ENFORCEMENT_ACTION_DATE" like \'__/__/2019\''

    waste_enforcements = get_data( sql, "ID_NUMBER" ) 
except EmptyDataError:
    print( "No data found")
waste_enforcements

,ACTIVITY_LOCATION,ENFORCEMENT_IDENTIFIER,ENFORCEMENT_TYPE,ENFORCEMENT_DESC,ENFORCEMENT_AGENCY,ENFORCEMENT_ACTION_DATE,PMP_AMOUNT,FMP_AMOUNT,FSC_AMOUNT,SCR_AMOUNT
ID_NUMBER,,,,,,,,,,
OKD072424104,OK,001,HQ110,VERBAL INFORMAL,S,03/28/2019,NaN,NaN,NaN,NaN
WVD107332223,WV,001,HQ120,WRITTEN INFORMAL,S,04/04/2019,NaN,NaN,NaN,NaN
WAD980722821,WA,001,HQ120,WRITTEN INFORMAL,S,05/31/2019,NaN,NaN,NaN,NaN
HIR000000067,HI,001,HI145,INSPECTION REPORT WRITTEN,S,03/29/2019,NaN,NaN,NaN,NaN
AZD980818165,AZ,42,AZ115,NOC/NOTICE OF OPPORTUNITY TO CORRECT,S,05/16/2019,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
RID980914550,RI,001,HQ120,WRITTEN INFORMAL,S,12/03/2019,NaN,NaN,NaN,NaN
GAD075876623,GA,001,HQ310,FINAL 3008(A) COMPLIANCE ORDER,E,02/19/2019,NaN,14500.0,NaN,NaN
SCD982156531,SC,001,HQ120,WRITTEN INFORMAL,S,04/11/2019,NaN,NaN,NaN,NaN


In [ ]:
# Number of enforcement actions each year per violation
waste_enforcements_metric = formatter(waste_enforcements.shape[0] / waste_violations.shape[0])
enforcements["RCRA"] = waste_enforcements_metric
display(HTML("<h3>"+waste_enforcements_metric+" enforcement actions per violation</h3>"))

In [ ]:
waste_enforcements["ENFORCEMENT_DESC"].unique()

array(['VERBAL INFORMAL', 'WRITTEN INFORMAL', 'INSPECTION REPORT WRITTEN',
       'NOC/NOTICE OF OPPORTUNITY TO CORRECT',
       '30-DAY COMPLIANCE WARNING LETTER', 'RETURN TO COMPLIANCE LETTER',
       'WARNING LETTER',
       'FACILITY ENFORCEMENT MEETING 3008(a) (COMPLIANCE SETTLEMENT)',
       'SUMMARY OF VIOLATIONS', 'RELEASE OF NOTICE OF VIOLATION',
       'FINAL CIVIL JUDICIAL ACTION FOR COMPLIANCE AND/OR MONETARY PENALTY',
       'FINAL 3008(A) COMPLIANCE ORDER',
       'Director-Division Warning Letter', 'LETTER OF WARNING',
       'RECEIVED FACILITY DOCUMENTATION',
       'INITIAL CIVIL JUDICIAL ACTION FOR COMPLIANCE AND/OR MONETARY PENALTY',
       'INITIAL PROPOSED CONSENT ORDER', 'NOTICE OF NONCOMPLIANCE LETTER',
       'DEP MEETING', 'INFORMAL VERBAL ENFORCEMENT',
       'INITIAL IMMINENT AND SUBSTANTIAL ENDANGERMENT ORDER',
       'COMPLIANCE ADVISORY', 'INITIAL 3008(A) COMPLIANCE',
       'ENFORCEMENT REFERRAL', 'SINGLE SITE CA/FO', 'Notice of Violation',
       'LETTER

In [ ]:
# Penalties each year per penalized facility (per penalty?)
waste_penalties = waste_enforcements.loc[waste_enforcements["FMP_AMOUNT"]>0]
waste_penalties_metric = formatter(sum(waste_penalties["FMP_AMOUNT"]) / len(waste_penalties.index.unique())) #Divide by penalized facilities
penalties["RCRA"] = waste_penalties_metric
display(HTML("<h3>$"+waste_penalties_metric +" per facility</h3>"))

## Greenhouse Gas Emissions in 2018 (latest data available)

In [ ]:
# Find facilities with pollutant exceedences
ghg_emissions = None
try:
    sql = 'select * from "POLL_RPT_COMBINED_EMISSIONS" where "REPORTING_YEAR" = \'2018\' and "PGM_SYS_ACRNM" = \'E-GGRT\''

    ghg_emissions = get_data( sql) 
except EmptyDataError:
    print( "No data found")
ghg_emissions

,REPORTING_YEAR,REGISTRY_ID,PGM_SYS_ACRNM,PGM_SYS_ID,POLLUTANT_NAME,ANNUAL_EMISSION,UNIT_OF_MEASURE,NEI_TYPE,NEI_HAP_VOC_FLAG
0,2018,110005826927,E-GGRT,1011331,Carbon dioxide,0.800,MTCO2e,NaN,NaN
1,2018,110058380130,E-GGRT,1011276,Methane,12045.250,MTCO2e,NaN,NaN
2,2018,110058380130,E-GGRT,1011276,Nitrous oxide,7.450,MTCO2e,NaN,NaN
3,2018,110058380130,E-GGRT,1011276,Carbon dioxide,15200.200,MTCO2e,NaN,NaN
4,2018,110058380087,E-GGRT,1011274,Carbon dioxide,27267.900,MTCO2e,NaN,NaN
...,...,...,...,...,...,...,...,...,...
21680,2018,110021138348,E-GGRT,1001372,Nitrous oxide,1392.554,MTCO2e,NaN,NaN
21681,2018,110033177425,E-GGRT,1003450,Methane,41518.500,MTCO2e,NaN,NaN
21682,2018,110006134691,E-GGRT,1006543,Carbon dioxide,1609796.300,MTCO2e,NaN,NaN
21683,2018,110007362086,E-GGRT,1001381,Carbon dioxide,97444.500,MTCO2e,NaN,NaN


In [ ]:
# Emissions each year per facility
ghg_emissions_metric = formatter(np.nansum(ghg_emissions["ANNUAL_EMISSION"]) / len(ghg_emissions["REGISTRY_ID"].unique())) #Divide by reporting facility
emissions["GHG"] = ghg_emissions_metric
display(HTML("<h3>"+ghg_emissions_metric+" MTCO2e (metric tons of carbon dioxide equivalent) emissions per reporting facility</h3>"))

# Data Export

In [ ]:
data = [inspections,
violations,
enforcements,
penalties,
emissions]

units = ["#inspections per 1000",
"#violations per 1000",
"#actions per violating facility",
"$ per penalized facility",
"amount of emissions (metric tons)"]

short_units = ["inspectionsper1000",
"violationsper1000",
"enforcementsperviolatingfacility",
"penaltiesperpenalizedfacility",
"emissions2018"]

for index, program in enumerate(data):
    # create dataframe
    df = pd.DataFrame(program, index=[0]).T
    df = df.rename(columns={0: units[index]})
    filename= short_units[index]+"_All_USA_pg4_081120.csv" #active-facilities_All_MA-CD4_3b_080620.csv
    df.to_csv(filename)
    print(df)

     #inspections per 1000
CAA                    261
CWA                 148.49
RCRA                 37.85
     #violations per 1000
CAA                     4
CWA                414.01
RCRA                24.31
     #actions per violating facility
CAA                                3
CWA                             0.03
RCRA                            0.48
     $ per penalized facility
CAA                     49279
CWA                  58130.57
RCRA                 33627.15
    amount of emissions (metric tons)
GHG                         421571.11
